In [2]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

In [3]:
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
autoreload JobOffers,dataframeHelpers

In [20]:
# Only run this if you want to redo the translations
# Start with google and use easynmt if it fails
JobOffers.writeTranslatedJobs(1000, method="google", fallback=True)

Failed to translate entry 201
Error was: the JSON object must be str, bytes or bytearray, not NoneType
Trying instead with EasyNMT
Failed to translate entry 202
Error was: 'Translator' object has no attribute 'raise_Exception'
Trying instead with EasyNMT
Failed to translate entry 211
Error was: 'Translator' object has no attribute 'raise_Exception'
Trying instead with EasyNMT
Failed to translate entry 214
Error was: 'Translator' object has no attribute 'raise_Exception'
Trying instead with EasyNMT
Failed to translate entry 218
Error was: 'Translator' object has no attribute 'raise_Exception'
Trying instead with EasyNMT
Failed to translate entry 222
Error was: the JSON object must be str, bytes or bytearray, not NoneType
Trying instead with EasyNMT
Failed to translate entry 222
Error was: the JSON object must be str, bytes or bytearray, not NoneType
Trying instead with EasyNMT
Failed to translate entry 279
Error was: the JSON object must be str, bytes or bytearray, not NoneType
Trying i

In [10]:
# rawOffers = JobOffers.readJsonFile("Data/output_v1.1.json")

In [21]:
translatedOffers = JobOffers.readJsonFile("Data/output_v1.1.translate.json")

In [22]:
jobsDf = dataframeHelpers.buildJobsDF()

In [23]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...
1,10814567,21420,[We are a prestigious and successful engineeri...,Wir sind eine renommierte und erfolgreiche Ing...
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,Bill Weyermann Partner AG erbringt seit über\n...
3,12473085,62100,[Administration Communale de Valderuzforstarti...,Administration communale de Val-de-RuzForstwar...
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,Le groupe Straumann connaît un succès internat...
...,...,...,...,...
395,18721310,31410;24210,[Lonza is one of the worlds leading and mosttr...,<p>Lonza is one of the world’s leading and mos...
396,18721311,24210,[Lonza is one of the worlds leading and mosttr...,<p>Lonza is one of the world’s leading and mos...
397,18721316,24210,[],
398,18722421,24210,[Unlimit Yourself Are you up for a challenge D...,Unlimit Yourself Are you up for a challenge? D...


In [48]:
#jobsDf.to_csv("Data/AnnotatedJobs.csv")

In [116]:
annotatedJobs200 = pd.read_csv("Data/PreliminaryResultsUSE_MJ.csv", sep=";")
annotatedJobs200.rename(columns={"ENVIRONMENTAL IDENTITY\nMJ" : "GREEN_IDENTITY_MJ", "GREEN_IDENTITY" : "GREEN_IDENTITY_EMILY"}, inplace=True)


In [117]:
annotatedJobs200['GREEN_IDENTITY_MJ'] = annotatedJobs200['GREEN_IDENTITY_MJ'].replace(['FALSE', 'FAUX ', '?'] , False)
annotatedJobs200['GREEN_IDENTITY_MJ'] = annotatedJobs200['GREEN_IDENTITY_MJ'].replace(['TRUE'], True)
annotatedJobs200['GREEN_IDENTITY_MJ'] = annotatedJobs200['GREEN_IDENTITY_MJ'].replace(['FAUX'] , False)
annotatedJobs200 = annotatedJobs200.drop(annotatedJobs200[annotatedJobs200['GREEN_IDENTITY_MJ'] == "same offer as above"].index)

In [118]:
annotatedJobs200[annotatedJobs200["GREEN_IDENTITY_MJ"].isna()] = False

In [161]:
annotatedJobs200[(annotatedJobs200["GREEN_IDENTITY_MJ"] != False) & (annotatedJobs200["GREEN_IDENTITY_MJ"] != True)]

,JOB_ID,ISCO,ORIGINAL_CONTENT,ENGLISH_JOB_CONTENT,TOP_ACTIVITY_MATCHES,TOP_ACTIVITY_IDS,BEST_ACTIVITY_MATCH_3SENT,BEST_ACTIVITY_ID_3SENT,BEST_ACTIVITY_MATCH_1SENT,BEST_ACTIVITY_ID_1SENT,...,JOB\nwith environemental or sustainability concerns,COMPANY\nwith environemental or sustainability concerns,"In the future, the job should include (more) environmental concerns",Important activities / skills in environmental jobs,Details,key words (for MJ),Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24


In [122]:
# This is all the entries where my classificationn and MJ's classification differ
disagreeDf = annotatedJobs200[annotatedJobs200["GREEN_IDENTITY_MJ"] != annotatedJobs200["GREEN_IDENTITY_EMILY"].astype('bool')]

In [162]:
disagreeSimpleDf = disagreeDf.loc[:,("JOB_ID", "ISCO", "ORIGINAL_CONTENT", "ENGLISH_JOB_CONTENT", "GREEN_IDENTITY_EMILY", "GREEN_IDENTITY_MJ", )]
disagreeSimpleDf

,JOB_ID,ISCO,ORIGINAL_CONTENT,ENGLISH_JOB_CONTENT,GREEN_IDENTITY_EMILY,GREEN_IDENTITY_MJ
0,13763695,21650,21.01.2017|LMT SA|100%\n\nUn Groupe leader dan...,21 01 2017LMT SA100 A leading group in the f...,True,False
2,12220025,21420,<p>Wir sind ein erfolgreiches und unabhängiges...,We are a successful and independent engineerin...,True,False
22,14624002,21650;31180,<h2>Dessinateur (-trice) en génie civil ou géo...,Draftsman in Civil Engineering or Geometer 60 ...,True,False
23,12723029,21420;21610,Bauingenieur/Architekt für den Bereich Baumeth...,Civil EngineeringArchitect For the area of,True,False
41,14536882,24210,"„Raffiniert veredelt""\n\n\nWir sind national w...",Refined refines we are national as internation...,False,True
75,11969310,31320,Chemikalienversorgung\nSind Sie Umomtivwierte ...,Chemicals Supply You are reversed Ultdte FLCEX...,False,True
89,13762077,24210,20.01.2017|Immoroc SA|60 bis 80%\n\nLe groupe ...,20 01 2017IMMOROC SA60 BIS 80 The PACI group...,False,True
95,13922235,62100,COMMUNE DE BROC\n\nMise au concours du poste d...,BRAC COMMON COMPETITION OF FORESTRIER TRANSTRU...,True,False
122,13946158,31120,Technicien en Bâtiment\n\nTechnicien en Bâtime...,Construction Technician Building Technician Yo...,True,False
123,13989853,24210,Supplier Quality Engineer\n\nS upplier Quality...,Supplier Quality Engineer S upplier Quality E...,False,True


In [134]:
disagreeSimpleDf.to_csv("Data/EmilyMJDifferentClassifications.csv")

In [137]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT,GREEN_IDENTITY_MJ
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...,0.0
1,10814567,21420,[We are a prestigious and successful engineeri...,Wir sind eine renommierte und erfolgreiche Ing...,0.0
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,Bill Weyermann Partner AG erbringt seit über\n...,0.0
3,12473085,62100,[Administration Communale de Valderuzforstarti...,Administration communale de Val-de-RuzForstwar...,1.0
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,Le groupe Straumann connaît un succès internat...,1.0
...,...,...,...,...,...
195,13761500,24210,[SA of the Horlogerie Manufacture Audemars Pig...,SA de la Manufacture d'horlogerie Audemars Pig...,1.0
196,13760809,21420,"[20, 01, 2017Henauer Gugler AGbern80 to 100 ...",20.01.2017|Henauer Gugler AG|Bern|80 bis 100%\...,0.0
197,14379272,24210,[Your tasks processing deviations conducting i...,"Ihre Aufgaben: - Bearbeitung von Abweichungen,...",1.0
198,14384238,21620,[Text building block header_1 Jobbörse Text mo...,"Textbaustein ""HEADER_1"" Jobbörse (Textbaustein...",1.0


In [138]:
jobsDf["GREEN_IDENTITY_MJ"] = annotatedJobs200["GREEN_IDENTITY_MJ"].astype('float32')
jobsDf["FIRST_SENTENCE"] = [x[0] for x in jobsDf["JOB_CONTENT"]]
jobsDf["ALL_SENTENCES"] = [". ".join(x) for x in jobsDf["JOB_CONTENT"]]

In [139]:
# Only take 1/3 of the "not green" jobs so you have a balanced data set
trainDf = jobsDf[((jobsDf["GREEN_IDENTITY_MJ"] == 0) & (jobsDf.index % 3 == 0)) | (jobsDf["GREEN_IDENTITY_MJ"] == 1.)]

In [140]:
trainDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT,GREEN_IDENTITY_MJ,FIRST_SENTENCE,ALL_SENTENCES
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...,0.0,We are a successful and independent engineerin...,We are a successful and independent engineerin...
3,12473085,62100,[Administration Communale de Valderuzforstarti...,Administration communale de Val-de-RuzForstwar...,1.0,Administration Communale de Valderuzforstartin...,Administration Communale de Valderuzforstartin...
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,Le groupe Straumann connaît un succès internat...,1.0,The Straumann Group is experiencing internatio...,The Straumann Group is experiencing internatio...
5,12523090,61300,[Le Devens Armée du Salutland Wirtin Efzstaubi...,"Le Devens, Armée du SalutLandwirt/in EFZSt-Aub...",1.0,Le Devens Armée du Salutland Wirtin Efzstaubin...,Le Devens Armée du Salutland Wirtin Efzstaubin...
6,11449790,21420;24310,[Civil Engineer as product manager SYTEC Bausy...,Bauingenieur/-in als Produktmanager\n\n\nSYTEC...,0.0,Civil Engineer as product manager SYTEC Bausys...,Civil Engineer as product manager SYTEC Bausys...
...,...,...,...,...,...,...,...
190,13763362,21650,"[LMT SA Mr, Cédric Jaccaud Road from Bettens ...",LMT SA\n\n\nMonsieur Cédric Jaccaud\nRoute de ...,1.0,LMT SA Mr,LMT SA Mr. Cédric Jaccaud Road from Bettens 1...
192,13785679,24210,[The Assurance Office for the Canton of Vaud h...,L'Office de l'assurance-invalidité pour le can...,0.0,The Assurance Office for the Canton of Vaud ha...,The Assurance Office for the Canton of Vaud ha...
195,13761500,24210,[SA of the Horlogerie Manufacture Audemars Pig...,SA de la Manufacture d'horlogerie Audemars Pig...,1.0,SA of the Horlogerie Manufacture Audemars Pigu...,SA of the Horlogerie Manufacture Audemars Pigu...
197,14379272,24210,[Your tasks processing deviations conducting i...,"Ihre Aufgaben: - Bearbeitung von Abweichungen,...",1.0,Your tasks processing deviations conducting in...,Your tasks processing deviations conducting in...


In [142]:
trainDf["FIRST_SENTENCE"][0]

'We are a successful and independent engineering office in Schönried'

In [143]:
from keras import layers,Model
import keras

In [144]:
model = tf.keras.models.Sequential()

In [145]:
model.add(hub.KerasLayer('universal-sentence-encoder_4',
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=True))

In [146]:
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [147]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 256,929,409
Trainable params: 256,929,409
Non-trainable params: 0
_________________________________________________________________


In [148]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [151]:
# Probably over trained
model.fit(trainDf["ALL_SENTENCES"], 
          trainDf["GREEN_IDENTITY_MJ"].astype("bool"), 
          epochs=3)

Epoch 1/3
4/4 [==============================] - 11s 1s/step - loss: 0.6932 - accuracy: 0.5192
Epoch 2/3
4/4 [==============================] - 6s 1s/step - loss: 0.6597 - accuracy: 0.6731
Epoch 3/3
4/4 [==============================] - 6s 1s/step - loss: 0.6303 - accuracy: 0.7500


In [152]:
jobsDf["GREEN_PREDICT"] = model.predict(jobsDf["ALL_SENTENCES"]) > 0.5

In [153]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,ORIGINAL_CONTENT,GREEN_IDENTITY_MJ,FIRST_SENTENCE,ALL_SENTENCES,GREEN_PREDICT
0,12220025,21420,[We are a successful and independent engineeri...,<p>Wir sind ein erfolgreiches und unabhängiges...,0.0,We are a successful and independent engineerin...,We are a successful and independent engineerin...,False
1,10814567,21420,[We are a prestigious and successful engineeri...,Wir sind eine renommierte und erfolgreiche Ing...,0.0,We are a prestigious and successful engineerin...,We are a prestigious and successful engineerin...,False
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,Bill Weyermann Partner AG erbringt seit über\n...,0.0,Bill Weyermann Partner AG has been producing e...,Bill Weyermann Partner AG has been producing e...,False
3,12473085,62100,[Administration Communale de Valderuzforstarti...,Administration communale de Val-de-RuzForstwar...,1.0,Administration Communale de Valderuzforstartin...,Administration Communale de Valderuzforstartin...,True
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,Le groupe Straumann connaît un succès internat...,1.0,The Straumann Group is experiencing internatio...,The Straumann Group is experiencing internatio...,True
...,...,...,...,...,...,...,...,...
195,13761500,24210,[SA of the Horlogerie Manufacture Audemars Pig...,SA de la Manufacture d'horlogerie Audemars Pig...,1.0,SA of the Horlogerie Manufacture Audemars Pigu...,SA of the Horlogerie Manufacture Audemars Pigu...,True
196,13760809,21420,"[20, 01, 2017Henauer Gugler AGbern80 to 100 ...",20.01.2017|Henauer Gugler AG|Bern|80 bis 100%\...,0.0,20,20. 01. 2017Henauer Gugler AGbern80 to 100 o...,False
197,14379272,24210,[Your tasks processing deviations conducting i...,"Ihre Aufgaben: - Bearbeitung von Abweichungen,...",1.0,Your tasks processing deviations conducting in...,Your tasks processing deviations conducting in...,True
198,14384238,21620,[Text building block header_1 Jobbörse Text mo...,"Textbaustein ""HEADER_1"" Jobbörse (Textbaustein...",1.0,Text building block header_1 Jobbörse Text mod...,Text building block header_1 Jobbörse Text mod...,True


In [159]:
len(jobsDf[jobsDf["GREEN_PREDICT"] & ~jobsDf["GREEN_IDENTITY_MJ"].astype("bool")])

43

In [157]:
#trainDf["GREEN_PREDICT"] = model.predict(trainDf["FIRST_SENTENCE"]) > 0.5

<ipython-input-157-a752a3e1c4cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDf["GREEN_PREDICT"] = model.predict(trainDf["FIRST_SENTENCE"]) > 0.5


In [12]:
pd.read_csv(os.path.expanduser("~/Desktop/Thesis/Data/Skills.csv"), sep=";", error_bad_lines=False)

,Category,Skill,Original ID,ID
0,Management of Financial Resources,Determining how money will be spent to get the...,300,1
1,Management of Material Resources,Obtaining and seeing to the appropriate use of...,301,2
2,Management of Personnel Resources —,"Motivating, developing, and directing people a...",302,3
3,Time Management —,Managing one's own time and the time of others.,303,4
4,Active Learning —,Understanding the implications of new informat...,305,5
...,...,...,...,...
69,Production and Processing,"Knowledge of raw materials, production process...",385,70
70,Public Safety and Security,"Knowledge of relevant equipment, policies, pro...",386,71
71,Sales and Marketing,Knowledge of principles and methods for showin...,387,72
72,Sociology and Anthropology,"Knowledge of group behavior and dynamics, soci...",388,73
